In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
from matplotlib import cm
from sklearn.ensemble import IsolationForest
import seaborn as sns
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from sklearn.covariance import EllipticEnvelope
from sklearn import svm
from catboost import CatBoostRegressor as cbr

%matplotlib inline

In [33]:
data_file = "dorothea_train.data"

# Delimiter
data_file_delimiter = '\s+'

largest_column_count=770
# Generate column names (will be 0, 1, 2, ..., largest_column_count - 1)
column_names = [i for i in range(0, largest_column_count)]

# Read csv
test = pd.read_csv(data_file, header=None, delimiter=data_file_delimiter, names=column_names)
test.head()

,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,768,769
0,191,367,614,634,711,1202,1220,1311,1472,1730,...,98959.0,99116.0,99187.0,99257.0,99321.0,99349.0,99580.0,99678.0,99686.0,99998.0
1,118,307,367,478,505,512,807,878,939,1024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,184,284,297,320,375,445,588,658,1108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,87,149,433,704,711,892,988,1056,1070,1234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,84,118,279,316,435,505,584,629,849,1029,...,96294.0,96504.0,96513.0,96640.0,96789.0,96907.0,96911.0,97143.0,97271.0,97496.0


In [34]:
print("Test:", test.shape)

Test: (800, 770)


In [35]:
test.isnull().sum().sum()

11168

In [36]:
test=test.dropna(axis=1, how='any', thresh=None, subset=None, inplace=False)
test.shape

(800, 657)

In [37]:
test.isnull().sum().sum()

0

In [38]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,647,648,649,650,651,652,653,654,655,656
0,191,367,614,634,711,1202,1220,1311,1472,1730,...,83741,83916,83938,84005,84062,84071,84416,84733,84737,84753
1,118,307,367,478,505,512,807,878,939,1024,...,90264,90440,90517,90548,90588,90688,90742,90851,90940,91046
2,10,184,284,297,320,375,445,588,658,1108,...,91593,91684,91737,91810,91943,92204,92346,92350,92381,92515
3,87,149,433,704,711,892,988,1056,1070,1234,...,88123,88389,88543,88567,89035,89091,89122,89173,89182,89643
4,84,118,279,316,435,505,584,629,849,1029,...,81457,81463,81728,81809,81881,82020,82154,82194,82382,82500


In [39]:
labels=pd.read_csv('dorothea_train.labels',header=None)
labels=labels.replace(-1,0)
labels

,0
0,0
1,0
2,0
3,0
4,0
...,...
795,0
796,0
797,0
798,0


In [40]:
test['labels'] = labels
test.head()

,0,1,2,3,4,5,6,7,8,9,...,648,649,650,651,652,653,654,655,656,labels
0,191,367,614,634,711,1202,1220,1311,1472,1730,...,83916,83938,84005,84062,84071,84416,84733,84737,84753,0
1,118,307,367,478,505,512,807,878,939,1024,...,90440,90517,90548,90588,90688,90742,90851,90940,91046,0
2,10,184,284,297,320,375,445,588,658,1108,...,91684,91737,91810,91943,92204,92346,92350,92381,92515,0
3,87,149,433,704,711,892,988,1056,1070,1234,...,88389,88543,88567,89035,89091,89122,89173,89182,89643,0
4,84,118,279,316,435,505,584,629,849,1029,...,81463,81728,81809,81881,82020,82154,82194,82382,82500,0


In [41]:
df=test
scaler = MinMaxScaler() 
num = scaler.fit_transform(df)
df = pd.DataFrame(num, columns = df.columns)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,648,649,650,651,652,653,654,655,656,labels
0,0.215909,0.351562,0.475240,0.378328,0.383815,0.629164,0.497818,0.499591,0.544735,0.466992,...,0.829042,0.828013,0.826577,0.826481,0.825748,0.829493,0.832916,0.831749,0.831586,0.0
1,0.132955,0.292969,0.277955,0.281734,0.264740,0.252321,0.317627,0.322567,0.333729,0.264351,...,0.902690,0.902191,0.900169,0.899846,0.900089,0.900567,0.901664,0.901386,0.902209,0.0
2,0.010227,0.172852,0.211661,0.169659,0.157803,0.177499,0.159686,0.204007,0.222486,0.288462,...,0.916733,0.915946,0.914363,0.915079,0.917121,0.918589,0.918509,0.917564,0.918694,0.0
3,0.097727,0.138672,0.330671,0.421672,0.383815,0.459858,0.396597,0.395339,0.385590,0.324627,...,0.879537,0.879934,0.877888,0.882387,0.882147,0.882366,0.882808,0.881650,0.886464,0.0
4,0.094318,0.108398,0.207668,0.181424,0.224277,0.248498,0.220332,0.220769,0.298100,0.265786,...,0.801350,0.803096,0.801878,0.801963,0.802705,0.804078,0.804385,0.805310,0.806302,0.0


In [42]:
df2=df

In [43]:
df2.shape

(800, 658)

In [44]:
from scipy import stats

In [45]:
clf = LocalOutlierFactor(n_neighbors=5, contamination=.1)
y_pred2 = clf.fit_predict(df2)
y_pred2=pd.Series(y_pred2).replace([-1,1],[1,0])

C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [46]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(df2.iloc[:,:657], df2.iloc[:,657:], train_size=0.8)

C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [47]:
from sklearn import svm, metrics
svm3_clf = svm.SVC(gamma=0.001, C=10)
svm3_clf.fit(X_train, y_train)
expected = y_validation
predicted = svm3_clf.predict(X_validation)
print(metrics.classification_report(expected, predicted))

             precision    recall  f1-score   support

        0.0       0.90      1.00      0.95       144
        1.0       0.00      0.00      0.00        16

avg / total       0.81      0.90      0.85       160



C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [48]:
df3=df2[y_pred2==0]
df3.head()

,0,1,2,3,4,5,6,7,8,9,...,648,649,650,651,652,653,654,655,656,labels
0,0.215909,0.351562,0.475240,0.378328,0.383815,0.629164,0.497818,0.499591,0.544735,0.466992,...,0.829042,0.828013,0.826577,0.826481,0.825748,0.829493,0.832916,0.831749,0.831586,0.0
1,0.132955,0.292969,0.277955,0.281734,0.264740,0.252321,0.317627,0.322567,0.333729,0.264351,...,0.902690,0.902191,0.900169,0.899846,0.900089,0.900567,0.901664,0.901386,0.902209,0.0
2,0.010227,0.172852,0.211661,0.169659,0.157803,0.177499,0.159686,0.204007,0.222486,0.288462,...,0.916733,0.915946,0.914363,0.915079,0.917121,0.918589,0.918509,0.917564,0.918694,0.0
3,0.097727,0.138672,0.330671,0.421672,0.383815,0.459858,0.396597,0.395339,0.385590,0.324627,...,0.879537,0.879934,0.877888,0.882387,0.882147,0.882366,0.882808,0.881650,0.886464,0.0
4,0.094318,0.108398,0.207668,0.181424,0.224277,0.248498,0.220332,0.220769,0.298100,0.265786,...,0.801350,0.803096,0.801878,0.801963,0.802705,0.804078,0.804385,0.805310,0.806302,0.0


In [49]:
X_train, X_validation, y_train, y_validation = train_test_split(df3.iloc[:,:657], df3.iloc[:,657:], train_size=0.8,random_state=42)
svm1_clf = svm.SVC(gamma=0.0001, C=10)
svm1_clf.fit(X_train, y_train)
expected = y_validation
predicted = svm1_clf.predict(X_validation)
print(metrics.classification_report(expected, predicted))

             precision    recall  f1-score   support

        0.0       0.93      1.00      0.96       134
        1.0       0.00      0.00      0.00        10

avg / total       0.87      0.93      0.90       144



C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [50]:
from sklearn.ensemble.forest import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=1000)
rf_clf.fit(X_train, y_train)
expected = y_validation
predicted = rf_clf.predict(X_validation)
print(metrics.classification_report(expected, predicted))

C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


             precision    recall  f1-score   support

        0.0       0.93      0.99      0.96       134
        1.0       0.00      0.00      0.00        10

avg / total       0.87      0.92      0.89       144



In [51]:
from sklearn.neighbors.classification import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=100, weights='distance')
knn_clf.fit(X_train, y_train)
expected = y_validation
predicted = knn_clf.predict(X_validation)
print(metrics.classification_report(expected, predicted))

             precision    recall  f1-score   support

        0.0       0.93      1.00      0.96       134
        1.0       0.00      0.00      0.00        10

avg / total       0.87      0.93      0.90       144



C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [52]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

In [53]:
mnb_clf = MultinomialNB()
mnb_clf.fit(X_train, y_train)
expected = y_validation
predicted = mnb_clf.predict(X_validation)
print(metrics.classification_report(expected, predicted))

             precision    recall  f1-score   support

        0.0       0.93      1.00      0.96       134
        1.0       0.00      0.00      0.00        10

avg / total       0.87      0.93      0.90       144



C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [54]:
gnb_clf = GaussianNB()
gnb_clf.fit(X_train, y_train)
expected = y_validation
predicted = gnb_clf.predict(X_validation)
print(metrics.classification_report(expected, predicted))

             precision    recall  f1-score   support

        0.0       0.93      0.87      0.90       134
        1.0       0.06      0.10      0.07        10

avg / total       0.87      0.82      0.84       144



C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [55]:
bnb_clf = BernoulliNB()
bnb_clf.fit(X_train, y_train)
expected = y_validation
predicted = bnb_clf.predict(X_validation)
print(metrics.classification_report(expected, predicted))

             precision    recall  f1-score   support

        0.0       0.93      1.00      0.96       134
        1.0       0.00      0.00      0.00        10

avg / total       0.87      0.93      0.90       144



C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\moham\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
